In [1]:
import warnings
warnings.filterwarnings("ignore")

from typing import Optional, List, Union
from numpy.typing import NDArray

import numpy as np
import pandas as pd

from tsururu.strategies import StrategiesFactory
from tsururu.dataset import TSDataset


In [2]:
# for global and multivariate all time seires should be in one .csv file, divided by id 
# for local each time serie should be in its own .csv file also with id column 
df_path = "datasets/global/simulated_data_to_check.csv"

# Configure the features settings
columns_and_features_params = {
    "target": {
        "column": ["y"],
        "type": "continious",
        
        "drop_raw_feature": False,
                },
    
    "date": {
        "column": ["ds"],
        "type": "datetime",
        
        "drop_raw_feature": False,
    },
    "id": {
        "column": ["unique_id"],
        "type": "categorical",
        "drop_raw_feature": False,
    }
}

# Configure the model parameters
model_params = {
    "h": 12,
    "input_size": 24,
    "hidden_size": 8,
    "conv_hidden_size": 8,
    "learning_rate": 1e-3,
    "max_steps": 10,
    "freq": "D"
}

# Configure the validation parameters
validation_params = None

# Configure the strategies parameters
strategy_params = {
    "is_multivariate": True,  
    "strategy_name": "MIMOStrategy",
    "strategy_params": {
        "horizon": 12,
        "model_name": "AutoformerRegressor_NN",
        "model_params": model_params,
        "validation_params": validation_params,
    }
}

In [3]:
strategies_factory = StrategiesFactory()

In [5]:
df=pd.read_csv(df_path,sep=',')

In [7]:
dataset = TSDataset(
    data=df,
    columns_and_features_params=columns_and_features_params,
    history=12
)

strategy = strategies_factory[strategy_params]

In [9]:
ids, test, pred, fit_time, forecast_time, num_iterations = strategy.back_test(dataset, cv=1)

Seed set to 1


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 26.64it/s]


In [10]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[NDArray[np.floating]]] = None,
    y_pred: Optional[List[NDArray[np.floating]]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, NDArray[np.floating]]], idx: int
    ) -> List[Optional[Union[float, NDArray[np.floating]]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

In [11]:
get_results(cv=1, regime="local", y_true=test, y_pred=pred, ids=ids)

,y_true_1,y_pred_1
0,1988.0,1974.964844
1,1989.0,1975.761719
2,1990.0,1975.788818
3,1991.0,1975.814209
4,1992.0,1975.837158
...,...,...
115,10995.0,10975.967773
116,10996.0,10976.074219
117,10997.0,10976.197266
118,10998.0,10975.919922


In [12]:
df = pd.read_csv('datasets/global/simulated_data_to_check.csv')
df['ds'] = pd.to_datetime(df['ds'], format='%Y-%m-%d')

In [13]:
# df = df.iloc[:728*2]
# df1 = df.iloc[:500]
# df2 = df.iloc[728:728+500]
# df_train = pd.concat([df1,df2])

In [14]:
from tsururu.models import Estimator
from typing import Dict, Union
from neuralforecast import NeuralForecast

from neuralforecast.models import Autoformer
class AutoformerRegressor:
    def __init__(
        self,
        model_params: Dict[str, Union[str, int]],
        validation_params=None):
        
        self.model_params = model_params
        self.validation_params = validation_params
        self.model = Autoformer(**self.model_params)
        self.nf = NeuralForecast(models=[self.model]) 
        
        super().__init__()
    
    def fit(self, train_df):
        
        self.nf.fit(train_df, val_size=12)
        
    def predict(self, test_df):
        forecasts = self.nf.predict(static_df=test_df)
        return forecasts

In [15]:
model_params = {
    "h": 12,
    "input_size": 24,
    "hidden_size": 16,
    "conv_hidden_size": 32,
    "learning_rate": 1e-3,
    "max_steps": 10,
}

In [16]:
model = AutoformerRegressor(model_params)

Seed set to 1


TypeError: NeuralForecast.__init__() missing 1 required positional argument: 'freq'

In [ ]:
model.fit(df)

Epoch 49: 100%|██████████| 1/1 [00:00<00:00,  3.10it/s, v_num=109, train_loss_step=6.520, train_loss_epoch=6.520, valid_loss=6.400]


In [16]:
df.drop(columns=['y'], inplace=True)

In [17]:
model.predict(df)

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 13.47it/s]


,ds,Autoformer
unique_id,,
0,2022-09-27,2001.394043
0,2022-09-28,2002.609741
0,2022-09-29,2002.976685
0,2022-09-30,2003.308350
0,2022-10-01,2003.364136
...,...,...
9,2022-10-04,11001.050781
9,2022-10-05,10999.132812
9,2022-10-06,10996.492188


In [2]:
# for global and multivariate all time seires should be in one .csv file, divided by id 
# for local each time serie should be in its own .csv file also with id column 
df_path = "datasets/global/simulated_data_to_check.csv"

# Configure the features settings
columns_and_features_params = {
    "target": {
        "column": ["value"],
        "type": "continious",
        "features": {
            # "LagTransformer": {"lags": 1},
            # "LastKnownNormalizer": {
            #     "regime": "ratio",
            #     "transform_train": True,
            #     "transform_target": True
            # } 
        },
        "drop_raw_feature": False,
    },
    "date": {
        "column": ["date"],
        "type": "datetime",
        "features": {
            # "DateSeasonsGenerator": {
                # Use seasonality features from the date column as features with datetime lags
                # Possible values: [
                #    "y": year, "m": month, "d": day, 
                #    "wd": weekday, "doy": dayofyear,
                #    "hour": hour, "min": minute, "sec": second, 
                #    "ms": microsecond,  "ns": nanosecond
                # ]
                # "seasonalities": ['doy', 'm', 'wd'], 
                # Use date from target point to make datetime features
                # "from_target_date": True,
            # },
            # "LagTransformer": {"lags": 1}
        },
        "drop_raw_feature": False,
    },
    "id": {
        "column": ["id"],
        "type": "categorical",
        "drop_raw_feature": False,
    }
}

# Configure the model parameters
model_params = {
    "h": 12,
    "input_size": 24,
    "patch_len": 24,
    "hidden_size": 16,
    "learning_rate": 1e-3,
    "max_steps":500,
    "max_steps": 50
}

# Configure the validation parameters
validation_params = None
# {
#     "type": 'KFold',
#     "n_splits": 3,
# }

# Configure the strategies parameters
strategy_params = {
    "is_multivariate": True,  #Алина сказала?
    # possible values: 
    # [
    #     'RecursiveStrategy', 
    #     'DirectStrategy', 
    #     'DirRecStrategy', 
    #     'MIMOStrategy', 
    #     'FlatWideMIMOStrategy', 
    # ]
    "strategy_name": "MIMOStrategy",
    "strategy_params": {
        "horizon": 12,
        "model_name": "PatchTSTRegressor_NN",
        "model_params": model_params,
        "validation_params": validation_params,
        # "k": 1,
    }
}

In [23]:
from neuralforecast.models import Autoformer
model_params = {
    "h": 12,
    "input_size": 24,
    "hidden_size": 16,
    "conv_hidden_size": 32,
    "learning_rate": 1e-3,
    "max_steps": 10,
}
model = Autoformer(**model_params)

Seed set to 1


In [24]:
from neuralforecast import NeuralForecast
d = NeuralForecast(models = [model],freq ="D")

In [25]:
d?

Type:           NeuralForecast
String form:    <neuralforecast.core.NeuralForecast object at 0x7f98aefd0c50>
File:           ~/miniconda3/envs/project_ts/lib/python3.11/site-packages/neuralforecast/core.py
Docstring:      <no docstring>
Init docstring:
The `core.StatsForecast` class allows you to efficiently fit multiple `NeuralForecast` models
for large sets of time series. It operates with pandas DataFrame `df` that identifies series
and datestamps with the `unique_id` and `ds` columns. The `y` column denotes the target
time series variable.

Parameters
----------
models : List[typing.Any]
    Instantiated `neuralforecast.models`
    see [collection here](https://nixtla.github.io/neuralforecast/models.html).
freq : str
    Frequency of the data,
    see [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
local_scaler_type : str, optional (default=None)
    Scaler to apply per-serie to all features before fitting, wh